In [46]:
import pandas as pd


In [47]:
raw_oc_release = pd.read_excel('oc release 2.1 and 2.2.xlsx')  # specify the actual sheet name
raw_oc_release.head(3)

,UID,Name,OC chapter,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
0,I-31930,Create custom diagnostic data manually,[F-18439] Ecosystem / Diagnostic Requirements (New),OC_Draft_Review,NaN,NaN,Release 2.3,Family\nTaxi\nCarsharing\nAtomibilty Subscriptions\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,NaN,NaN
1,I-31923,Switch incoming calls,[F-7603] 05 Connectivity / Integration with messengers on the smartphone (New),OC_Draft_Review,[HMXSpec303-207319] Phone Functionality System (Ready for SA),NaN,Release 2.3,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat,NaN,NaN
2,I-31907,Showing the call device,[F-7603] 05 Connectivity / Integration with messengers on the smartphone (New),OC_Draft_Review,[HMXSpec303-207319] Phone Functionality System (Ready for SA),NaN,Release 2.3,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat,NaN,NaN


In [48]:
# Split the 'second_column' by new line
raw_oc_release['[SA_only] Link to Use Case'] = raw_oc_release['[SA_only] Link to Use Case'].str.split('\n')
pd.set_option('display.max_colwidth', None)
raw_oc_release.head(3)

,UID,Name,OC chapter,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
0,I-31930,Create custom diagnostic data manually,[F-18439] Ecosystem / Diagnostic Requirements (New),OC_Draft_Review,NaN,NaN,Release 2.3,Family\nTaxi\nCarsharing\nAtomibilty Subscriptions\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,NaN,NaN
1,I-31923,Switch incoming calls,[F-7603] 05 Connectivity / Integration with messengers on the smartphone (New),OC_Draft_Review,[HMXSpec303-207319] Phone Functionality System (Ready for SA),NaN,Release 2.3,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat,NaN,NaN
2,I-31907,Showing the call device,[F-7603] 05 Connectivity / Integration with messengers on the smartphone (New),OC_Draft_Review,[HMXSpec303-207319] Phone Functionality System (Ready for SA),NaN,Release 2.3,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat,NaN,NaN


In [49]:
# drop where [SA_only] Link to Use Case is empty
raw_oc_release.dropna(subset=['[SA_only] Link to Use Case'], inplace=True)
raw_oc_release.head(3)


,UID,Name,OC chapter,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),"[[U-25341] Issue: Control to calculate the SOC of power battery (Accepted), [U-26956] Issue: Display Energy Save Mode (Accepted), [U-27170] Issue: Control energy saving mode (Accepted), [U-29022] Issue: Prompt enable energy save mode (Accepted)]",Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
21,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),"[[U-25341] Issue: Control to calculate the SOC of power battery (Accepted), [U-26956] Issue: Display Energy Save Mode (Accepted), [U-27170] Issue: Control energy saving mode (Accepted), [U-29022] Issue: Prompt enable energy save mode (Accepted)]",Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
22,I-31527,Atom notifies the Driver and limits the speed when the door or hood is opened,[F-17922] 02 Opening and Closing / Doors (New),HMI Approved,[HMXSpec106-207306] Power Door Control System (SA in progress),"[[U-23675] Issue: 检测当前车门状态 / Determine Actual Door Status (Changed), [U-23957] Issue: 行车车门开启状态报警显示 / Display Door Open State Warning (Changed), [U-23958] Issue: 行车车门开启声音警示控制 / Notify of Door Open State (Changed), [U-23959] Issue: 车速开门警告 / Control Door Open State Warning (Accepted), [U-23960] Issue: 机罩实际状态判断 / Determine Actual Hood Status (Accepted), [U-23961] Issue: 机罩开启状态报警 / Control Hood Open State Warning (Accepted), [U-23962] Issue: 显示机罩开启报警状态 / Display Hood Open State Warning (Changed), [U-23963] Issue: 行驶时机罩开启状态警示 / Notify of Hood Open State (Changed), [U-29973] Issue: speed limit to 10km/h when opened door (Accepted)]",Release 2.3,Family\nTaxi\nCarsharing\nAtomibilty Subscriptions\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,NaN,"CERTIFICATION,CERTIFICATION,CERTIFICATION,CERTIFICATION,OTHER,OTHER,OTHER,OTHER,CERTIFICATION"


In [50]:
for index, row in raw_oc_release.iterrows():
    print(row['[SA_only] Link to Use Case'])

['[U-25341]  Issue: Control to calculate the SOC of power battery (Accepted)', '[U-26956]  Issue: Display Energy Save Mode (Accepted)', '[U-27170]  Issue: Control energy saving mode (Accepted)', '[U-29022]  Issue: Prompt enable energy save mode (Accepted)']
['[U-25341]  Issue: Control to calculate the SOC of power battery (Accepted)', '[U-26956]  Issue: Display Energy Save Mode (Accepted)', '[U-27170]  Issue: Control energy saving mode (Accepted)', '[U-29022]  Issue: Prompt enable energy save mode (Accepted)']
['[U-23675]  Issue: 检测当前车门状态 / Determine Actual Door Status (Changed)', '[U-23957]  Issue: 行车车门开启状态报警显示 / Display Door Open State Warning (Changed)', '[U-23958]  Issue: 行车车门开启声音警示控制 / Notify\xa0of Door Open State (Changed)', '[U-23959]  Issue: 车速开门警告 / Control Door Open State Warning (Accepted)', '[U-23960]  Issue: 机罩实际状态判断 / Determine Actual Hood Status (Accepted)', '[U-23961]  Issue: 机罩开启状态报警 / Control Hood Open State Warning (Accepted)', '[U-23962]  Issue: 显示机罩开启报警状态 / Display

In [51]:
# Use explode to duplicate the first column and expand the second column
oc_release = raw_oc_release.explode('[SA_only] Link to Use Case')

oc_release.head(5)

,UID,Name,OC chapter,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),[U-25341] Issue: Control to calculate the SOC of power battery (Accepted),Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),[U-26956] Issue: Display Energy Save Mode (Accepted),Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),[U-27170] Issue: Control energy saving mode (Accepted),Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),[U-29022] Issue: Prompt enable energy save mode (Accepted),Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
21,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),[U-25341] Issue: Control to calculate the SOC of power battery (Accepted),Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"


In [52]:
# Replace text in the column using regular expression to extract the pattern U-xxxxx
oc_release['[SA_only] Link to Use Case'] = oc_release['[SA_only] Link to Use Case'].str.extract(r'(U-\d+)')
oc_release_exploded = oc_release
oc_release_exploded.head(5)


,UID,Name,OC chapter,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-26956,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-27170,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
20,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-29022,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
21,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"


In [53]:
# Load the Excel files into dataframes
raw_fl = pd.read_excel('FL Release 2.1.1.xlsx', header=0)  # specify the actual sheet name
raw_fl

,UID,Caption,State,Link to Operational Concept UC,UCID,[VEHICLE] Functions Scope,FIP
0,U-25127,显示坡路保持控制功能状态/Display Hill hold control status,Accepted,[I-23569] Use Case CF: Hill Hold (Done after IAT),UC-205-11-05,-,Release A - No\nRelease 2.0 - Yes\nRelease 2.1 - Modified\nRelease 2.2 - Yes\nRelease 2.3 - Yes
1,U-3709,通过SWP Android切换AEB开启关闭 / Toggle AEB (Emergency Brake Assist) via SWP Android,Changed,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-01,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.1 Appendix - Yes\nRelease 2.2 - Modified\nRelease 2.3 - Yes
2,U-3710,控制制动预填充 / Control Brake Prefill,Accepted,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-15,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes
3,U-3712,控制刹停后驻车 / Control Parking after Braking,Accepted,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-04,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes
4,U-3714,控制AEB (自动紧急制动) / Control AEB (Automatic Emergency Braking),Accepted,[I-7092] Use Case CF: Tagging incidents during DVR recording (Done after IAT)\n[I-7205] Use Case CF: FCW Main Scenario (Done after IAT)\n[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-06,"-,-,-",Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes
...,...,...,...,...,...,...,...
2133,U-31676,Show available users internet traffic of MNO profile /显示当前运营商用户网络可用流量,Accepted,NaN,UC-1204-01-08,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes
2134,U-31677,Show available vehicle internet traffic of MNO profile /显示当前运营商OEM网络可用流量,Accepted,NaN,UC-1204-01-09,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes
2135,U-31678,Purchase additional users internet traffic of MNO profile /购买指定运营商用户流量,Accepted,NaN,UC-1204-01-10,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes
2136,U-31679,Allow user application to use internet traffic / 允许用户应用访问因特网,Accepted,NaN,UC-1204-01-11,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes


In [54]:
raw_fl['[SA_only] Link to Use Case'] = raw_fl['UID']
raw_fl

,UID,Caption,State,Link to Operational Concept UC,UCID,[VEHICLE] Functions Scope,FIP,[SA_only] Link to Use Case
0,U-25127,显示坡路保持控制功能状态/Display Hill hold control status,Accepted,[I-23569] Use Case CF: Hill Hold (Done after IAT),UC-205-11-05,-,Release A - No\nRelease 2.0 - Yes\nRelease 2.1 - Modified\nRelease 2.2 - Yes\nRelease 2.3 - Yes,U-25127
1,U-3709,通过SWP Android切换AEB开启关闭 / Toggle AEB (Emergency Brake Assist) via SWP Android,Changed,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-01,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.1 Appendix - Yes\nRelease 2.2 - Modified\nRelease 2.3 - Yes,U-3709
2,U-3710,控制制动预填充 / Control Brake Prefill,Accepted,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-15,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes,U-3710
3,U-3712,控制刹停后驻车 / Control Parking after Braking,Accepted,[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-04,-,Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes,U-3712
4,U-3714,控制AEB (自动紧急制动) / Control AEB (Automatic Emergency Braking),Accepted,[I-7092] Use Case CF: Tagging incidents during DVR recording (Done after IAT)\n[I-7205] Use Case CF: FCW Main Scenario (Done after IAT)\n[I-7202] Use Case CF: AEB Main Scenario (Done after IAT),UC-501-01-06,"-,-,-",Release A - Yes\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes,U-3714
...,...,...,...,...,...,...,...,...
2133,U-31676,Show available users internet traffic of MNO profile /显示当前运营商用户网络可用流量,Accepted,NaN,UC-1204-01-08,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes,U-31676
2134,U-31677,Show available vehicle internet traffic of MNO profile /显示当前运营商OEM网络可用流量,Accepted,NaN,UC-1204-01-09,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes,U-31677
2135,U-31678,Purchase additional users internet traffic of MNO profile /购买指定运营商用户流量,Accepted,NaN,UC-1204-01-10,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes,U-31678
2136,U-31679,Allow user application to use internet traffic / 允许用户应用访问因特网,Accepted,NaN,UC-1204-01-11,NaN,Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes,U-31679


In [55]:
# Ensure there is a common column to merge on, here assuming both have a 'UID' or similar
# Adjust the names if needed according to your DataFrame structures
raw_fl['[SA_only] Link to Use Case'] = raw_fl['UID']  # or another column that matches

# Merge data from raw_fl to oc_release based on the 'UID'
oc_release = oc_release.merge(raw_fl[['[SA_only] Link to Use Case', 'FIP']], on='[SA_only] Link to Use Case', how='left')

# Move the 'FIP' column to the 4th position
columns = list(oc_release.columns)
# Assuming index is zero-based, the 4th position is index 3
columns.insert(3, columns.pop(columns.index('FIP')))
oc_release = oc_release[columns]

# Display the modified DataFrame
oc_release.head()

,UID,Name,OC chapter,FIP,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
0,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Release A - No\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
1,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.2 - Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-26956,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
2,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.1 Appendix - No\nRelease 2.2 - Yes\nRelease 2.3 - No,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-27170,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
3,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Release A - No\nRelease 2.0 - No\nRelease 2.1 - Yes\nRelease 2.1 Appendix - No\nRelease 2.2 - Yes\nRelease 2.3 - No,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-29022,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
4,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),Release A - No\nRelease 2.0 - Yes\nRelease 2.1 - Yes\nRelease 2.2 - Yes\nRelease 2.3 - Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"


In [56]:
oc_release['FIP'] = oc_release['FIP'].str.split('\n')

In [57]:
oc_release

,UID,Name,OC chapter,FIP,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
0,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
1,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.2 - Yes]",Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-26956,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
2,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-27170,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
3,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-29022,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
4,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
...,...,...,...,...,...,...,...,...,...,...,...
1744,I-7088,Access to the microphone and the cameras for third-part applications installed on the SWP,[F-6871] 08 Цифровой видеорегистратор и безопасность / Digital Video Recorder (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - Yes, Release 2.2 - Yes]",Done after IAT,[HMXSpec304-207320] ATOM User Profile (IAM) (Ready for SA),U-27463,Release 2.2,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,-,"OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER"
1745,I-7088,Access to the microphone and the cameras for third-part applications installed on the SWP,[F-6871] 08 Цифровой видеорегистратор и безопасность / Digital Video Recorder (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - Yes, Release 2.2 - Yes]",Done after IAT,[HMXSpec304-207320] ATOM User Profile (IAM) (Ready for SA),U-27464,Release 2.2,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,-,"OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER"
1746,I-7088,Access to the microphone and the cameras for third-part applications installed on the SWP,[F-6871] 08 Цифровой видеорегистратор и безопасность / Digital Video Recorder (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - Yes, Release 2.2 - Yes]",Done after IAT,[HMXSpec304-207320] ATOM User Profile (IAM) (Ready for SA),U-27465,Release 2.2,Family\nTaxi\nCarsharing\nDelivery 1 seat\nDelivery 2 seat\nTaxi 3 seats,-,"OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER"
1747,I-7088,Access to the microphone and the cameras for third-part applications installed on the SWP,[F-6871] 08 Цифровой видеорегистратор и безопасность / Digital Video Recorder (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Ye

In [59]:
# Function to check if any of the criteria match
def check_fip_criteria(fip_list):
    if not isinstance(fip_list, list):
        return False
    # Convert all list elements to lowercase for consistent comparison
    fip_list_lower = [fip_item.lower() for fip_item in fip_list]
    # Define the criteria to look for
    criteria = [
        "release 2.1 - yes", "release 2.1 - modified", 
        "release 2.1 appendix - yes", "release 2.1 appendix - modified"
    ]
    # Check if any of the criteria are present in the list
    return any(criterion in fip_list_lower for criterion in criteria)

# Apply the check to each row in the FIP column
oc_release['Is FIP 2.1'] = oc_release['FIP'].apply(lambda x: 'Yes' if check_fip_criteria(x) else '')

# Insert the new column next to the FIP column
fip_index = oc_release.columns.get_loc('FIP')
oc_release.insert(fip_index + 1, 'Is FIP 2.1', oc_release.pop('Is FIP 2.1'))

# Display the DataFrame to verify the new column
oc_release.head()

,UID,Name,OC chapter,FIP,Is FIP 2.1,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base
0,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
1,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.2 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-26956,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
2,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-27170,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
3,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-29022,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"
4,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER"


In [62]:
# Assume oc_release is your existing DataFrame

# Function to determine if all values are "Yes" for a column in a grouped DataFrame
def all_yes(series):
    return 'Yes' if all(value == 'Yes' for value in series) else 'No'

# Group by 'UID' and apply the function to 'Is FIP 2.1'
is_oc_release_2_1 = oc_release.groupby('UID')['Is FIP 2.1'].apply(all_yes)

# Map the results back to your original DataFrame
oc_release['Is OC Release 2.1'] = oc_release['UID'].map(is_oc_release_2_1)

# Display the DataFrame to verify the new column
oc_release.head()

,UID,Name,OC chapter,FIP,Is FIP 2.1,UC Status,HMX specification,[SA_only] Link to Use Case,Product Release,Product Applicability,[OC] Functions Scope,[OC] Base,Is OC Release 2.1
0,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER",Yes
1,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.2 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-26956,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER",Yes
2,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-27170,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER",Yes
3,I-31548,Prompt the driver about turn on energy save mode when in low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - No, Release 2.1 - Yes, Release 2.1 Appendix - No, Release 2.2 - Yes, Release 2.3 - No]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-29022,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER",Yes
4,I-31547,Automatic enable energy save mode when in critical low SOC,[F-6878] 10 Charging and Energy Consumption (Battery Management) / High Voltage Energy Management (New),"[Release A - No, Release 2.0 - Yes, Release 2.1 - Yes, Release 2.2 - Yes, Release 2.3 - Yes]",Yes,Assign to IAT,[HMXSpec403-207335] Drive Mode Management Subsystem (Ready for SA),U-25341,Release 2.3,NaN,NaN,"VTA,OTHER,OTHER,OTHER",Yes


In [64]:
#export to excel
oc_release.to_excel("withfip.xlsx")

In [67]:
# Filter the DataFrame to find UIDs where 'Is OC Release 2.1' is 'Yes'
unique_uids_all_yes = oc_release[oc_release['Is OC Release 2.1'] == 'Yes']['UID'].unique()

# Convert the array of unique UIDs into a comma-separated string
unique_uids_all_yes_str = ', '.join(unique_uids_all_yes)

print("Candidates for Product Release 2.1:")
print("# of UCs:", len(unique_uids_all_yes), unique_uids_all_yes_str)

Candidates for Product Release 2.1:
# of UCs: 100 I-31548, I-31547, I-30915, I-30754, I-30750, I-30462, I-30414, I-30401, I-30370, I-29934, I-29808, I-29446, I-29302, I-29301, I-29126, I-26016, I-25994, I-25988, I-25519, I-25360, I-25356, I-25346, I-25342, I-25337, I-24694, I-24689, I-24432, I-7030, I-7026, I-7019, I-7016, I-7012, I-7009, I-7006, I-6996, I-6993, I-6992, I-6991, I-6985, I-6983, I-6979, I-6977, I-6976, I-6968, I-6963, I-6957, I-6952, I-6949, I-6948, I-6943, I-25246, I-31644, I-30954, I-30953, I-30951, I-30361, I-30327, I-30296, I-30285, I-30282, I-29979, I-29978, I-29961, I-29300, I-29167, I-29009, I-29007, I-28060, I-27748, I-26982, I-26447, I-26114, I-25813, I-25812, I-25806, I-25771, I-25755, I-25584, I-25076, I-24709, I-24457, I-24437, I-24413, I-24408, I-24407, I-24401, I-24389, I-23363, I-11478, I-11467, I-8625, I-8577, I-7157, I-7087, I-6607, I-30786, I-25501, I-25504, I-29035, I-26278
